In [30]:
# I exported the Weekly dataset from R Using write.csv(Default, "C:/Users/mcichra/Documents/ISLR work/Default.csv")

import pandas as pd
pd.set_option('display.float_format', '{:,.2f}'.format)
converters = {'a': int, 'b': int, 'c': int}
Default = pd.read_csv('Default.csv', converters=converters)
Default.shape

(10000, 5)

In [41]:
#remember statsmodels needs 0 and 1 for binary variables so need to convert the no's and yes's
Default['default2'] = Default.default.factorize()[0]
Default['student2'] = Default.student.factorize()[0]
Default.head(20)

,Unnamed: 0,default,student,balance,income,default2,student2
0,1,No,No,729.53,"44,361.63",0,0
1,2,No,Yes,817.18,"12,106.13",0,1
2,3,No,No,"1,073.55","31,767.14",0,0
3,4,No,No,529.25,"35,704.49",0,0
4,5,No,No,785.66,"38,463.50",0,0
5,6,No,Yes,919.59,"7,491.56",0,1
6,7,No,No,825.51,"24,905.23",0,0
7,8,No,Yes,808.67,"17,600.45",0,1
8,9,No,No,"1,161.06","37,468.53",0,0
9,10,No,No,0.00,"29,275.27",0,0


In [27]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn.linear_model as skl_lm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures

import statsmodels.api as sm
import statsmodels.formula.api as smf

from patsy import dmatrices, dmatrix #patsy library for r-style formula entry

%matplotlib inline
plt.style.use('seaborn-white')

In [32]:
np.random.seed(1)
train1 = np.random.choice(10000, 7000, replace=False)
DefaultTrain = Default.loc[Default.index.isin(train1)]
DefaultTest = Default.loc[~Default.index.isin(train1)]
glmfit = smf.logit(formula='default2 ~ income + balance', data = DefaultTrain).fit()
print(glmfit.summary())

Optimization terminated successfully.
         Current function value: 0.077448
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:               default2   No. Observations:                 7000
Model:                          Logit   Df Residuals:                     6997
Method:                           MLE   Df Model:                            2
Date:                Mon, 17 Sep 2018   Pseudo R-squ.:                  0.4474
Time:                        09:45:29   Log-Likelihood:                -542.14
converged:                       True   LL-Null:                       -981.14
                                        LLR p-value:                2.204e-191
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.3514      0.515    -22.060      0.000     -12.360     -10.343
income      1.847e-05   5.98

In [36]:
glmprobs = glmfit.predict(DefaultTest)
glmpred = np.repeat(0, 3000)
glmpred[glmprobs>.5] = 1
df_ = pd.DataFrame({'Actual Direction': DefaultTest.default2, 'Predicted Direction': glmpred})
df_.groupby(['Predicted Direction','Actual Direction']).size().unstack('Actual Direction')
np.mean(glmpred!=DefaultTest.default2)

0.029333333333333333

In [37]:
np.random.seed(1)
train2= np.random.choice(10000, 8000, replace=False)
DefaultTrain2 = Default.loc[Default.index.isin(train2)]
DefaultTest2 = Default.loc[~Default.index.isin(train2)]
glmfit2 = smf.logit(formula='default2 ~ income + balance', data = DefaultTrain2).fit()
glmprobs2 = glmfit2.predict(DefaultTest2)
glmpred2 = np.repeat(0, 2000)
glmpred2[glmprobs2>.5] = 1
df_ = pd.DataFrame({'Actual Direction': DefaultTest2.default2, 'Predicted Direction': glmpred2})
df_.groupby(['Predicted Direction','Actual Direction']).size().unstack('Actual Direction')
np.mean(glmpred2!=DefaultTest2.default2)

Optimization terminated successfully.
         Current function value: 0.079833
         Iterations 10


0.027

In [38]:
np.random.seed(1)
train3= np.random.choice(10000, 9000, replace=False)
DefaultTrain3 = Default.loc[Default.index.isin(train3)]
DefaultTest3 = Default.loc[~Default.index.isin(train3)]
glmfit3 = smf.logit(formula='default2 ~ income + balance', data = DefaultTrain3).fit()
glmprobs3 = glmfit3.predict(DefaultTest3)
glmpred3 = np.repeat(0, 1000)
glmpred3[glmprobs3>.5] = 1
df_ = pd.DataFrame({'Actual Direction': DefaultTest3.default2, 'Predicted Direction': glmpred3})
df_.groupby(['Predicted Direction','Actual Direction']).size().unstack('Actual Direction')
np.mean(glmpred3!=DefaultTest3.default2)

Optimization terminated successfully.
         Current function value: 0.080068
         Iterations 10


0.023

Validation set error appears to decrease as proportion of observations assigned to the training set increases.

In [50]:
np.random.seed(1)
train4 = np.random.choice(10000, 8000, replace=False)
DefaultTrain4 = Default.loc[Default.index.isin(train4)]
DefaultTest4 = Default.loc[~Default.index.isin(train4)]
glmfit4 = smf.logit(formula='default2 ~ income + balance + student', data = DefaultTrain4).fit()
glmprobs4 = glmfit4.predict(DefaultTest4)
glmpred4 = np.repeat(0, 2000)
glmpred4[glmprobs4>.5] = 1
df_ = pd.DataFrame({'Actual Direction': DefaultTest4.default2, 'Predicted Direction': glmpred4})
df_.groupby(['Predicted Direction','Actual Direction']).size().unstack('Actual Direction')
np.mean(glmpred4!=DefaultTest4.default2)

Optimization terminated successfully.
         Current function value: 0.079565
         Iterations 10


0.028000000000000001

Compared to my results in R the addition of the student variable appears to worsen the validation set error here. This occurs for all the training set sizes I've tried here. Probably has to do with the observations selected by the randomization here vs. those in R.